In [33]:
#Library installations
%pip install qBraid  # Install qBraid for quantum computing integration

# Import necessary libraries
import numpy as np  # Import NumPy for numerical operations
from qiskit import QuantumCircuit, transpile  # QuantumCircuit for building quantum circuits, transpile to optimize for device
from qiskit.visualization.counts_visualization import plot_histogram  # Import plot_histogram for visualizing results
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager  # Pass manager for circuit optimization

from qbraid.runtime.ibm import QiskitRuntimeProvider, QiskitJob  # Import qBraid-specific runtime functions for IBM quantum job execution

from qiskit_ibm_runtime import QiskitRuntimeService  # Import runtime service to handle IBM Quantum service
from dotenv import load_dotenv  # Load .env file for environment variables

import os  # Import OS for file and environment operations
import matplotlib.pyplot as plt  # Import matplotlib for plotting
import math  # Standard math library
import random  # Standard random number generation

Note: you may need to restart the kernel to use updated packages.


In [34]:
# Set up your API key
IBM_APIKEY = 'your_ibm_apikey'  # This is the unique key for accessing IBM Quantum services

# Save your IBM Quantum account for accessing devices
QiskitRuntimeService.save_account(channel='ibm_quantum', token=IBM_APIKEY, overwrite=True)

# Define the quantum random number generator circuit
def create_qrng_circuit(num_qubits):
    circ = QuantumCircuit(num_qubits)  # Create a quantum circuit with the specified number of qubits
    for i in range(num_qubits):
        circ.h(i)  # Apply Hadamard gate to each qubit to create superposition (essential for randomness)
    circ.measure_all()  # Measure all qubits after applying Hadamard gates
    return circ

# Function to generate pseudorandom number data (PRNG)
def generate_prng_data(num_samples, num_bits_per_sample):
    prng_data = []  # Initialize empty list for PRNG binary strings
    for _ in range(num_samples):
        bits = ''.join(np.random.choice(['0', '1'], size=num_bits_per_sample))  # Generate a random binary string
        prng_data.append(bits)  # Append generated string to the PRNG data list
    return prng_data

# Function to combine QRNG and PRNG data for training machine learning model
def combine_qrng_prng_data(qrng_data, prng_data):
    combined_data = []  # Initialize empty list to store combined data
    for binary_string in qrng_data:
        if len(binary_string) == len(qrng_data[0]):  # Ensure all binary strings are of the same length
            combined_data.append(f"{binary_string} 1")  # Label QRNG data with '1'
    for binary_string in prng_data:
        if len(binary_string) == len(prng_data[0]):  # Ensure all PRNG strings are consistent in length
            combined_data.append(f"{binary_string} 2")  # Label PRNG data with '2'
    return combined_data  # Return combined list of QRNG and PRNG data

# Function to launch a quantum job on a real IBM Quantum device
def launch_job(device_name, shot_count, circ):
    provider = QiskitRuntimeService()  # Initialize provider using saved IBM Quantum account
    print(provider.backends())  # Print available quantum devices
    device = provider.backend(device_name)  # Select the specified quantum device

    # Transpile the circuit for the device to optimize it for hardware execution
    transpiled_circuit = transpile(circ, device)

    # Run the transpiled circuit on the specified quantum device
    job = device.run(transpiled_circuit, shots=shot_count, memory=True)
    print(f"Job ID: {job.job_id()}")  # Output job ID for tracking
    return job.job_id()  # Return the job ID for future reference

# Function to retrieve job results from a previously submitted quantum job
def get_job_results(job_id, length):
    provider = QiskitRuntimeService()  # Use saved account to initialize the provider
    job = provider.job(job_id)  # Retrieve the job using the provided job ID

    # Get the job results
    result = job.result()
    rawData = result.get_memory()  # Get the raw bit results from the job memory

    # Flatten the raw data and group it into chunks of the specified length
    unbrokenData = [bit for shot in rawData for bit in shot]  # Flatten bitwise results
    data = [''.join(unbrokenData[i:i+length]) for i in range(0, len(unbrokenData), length)]  # Group bits into appropriate lengths
    
    return data  # Return the processed data

# Function to write combined QRNG and PRNG data to a text file
def write_data_to_file(data, filename):
    with open(filename, 'w') as file:  # Open file in write mode
        for entry in data:
            file.write(entry + '\n')  # Write each entry to the file, followed by a newline

# Function to concatenate QRNG data from multiple quantum jobs
def concatenate_qrng_data(job_ids, length):
    qrng_data = []  # Initialize empty list to store QRNG data
    for job_id in job_ids:
        data = get_job_results(job_id, length)  # Retrieve job results for each job ID
        qrng_data.extend(data)  # Append data to the QRNG data list
    return qrng_data  # Return concatenated QRNG data

# Main execution block
if __name__ == "__main__":
    # Set the total number of qubits for the quantum random number generator
    total_qubits = 500  # Define how many total qubits are needed for randomness
    max_device_qubits = 127  # Maximum qubit capacity of the quantum device
    num_shots = 100  # Define the number of shots (repetitions of the circuit)
    job_ids = []  # List to store job IDs from quantum device runs

    # Loop to break total qubits into chunks that the device can handle
    for i in range(0, total_qubits, max_device_qubits):
        num_qubits = min(max_device_qubits, total_qubits - i)  # Determine the number of qubits for the current chunk
        print(f"Running circuit with {num_qubits} qubits.")  # Inform the user about current circuit size

        # Create the quantum circuit with the specified number of qubits
        circ = create_qrng_circuit(num_qubits)

        # Launch the quantum job on the IBM Quantum device
        job_id = launch_job('ibm_brisbane', num_shots, circ)
        job_ids.append(job_id)  # Store the job ID for later retrieval

    # Concatenate all QRNG data from the executed jobs
    qrng_data = concatenate_qrng_data(job_ids, max_device_qubits)

    # Generate PRNG data to match the length of the QRNG data
    prng_data = generate_prng_data(len(qrng_data), len(qrng_data[0]))

    # Combine the QRNG and PRNG data for training purposes
    combined_data = combine_qrng_prng_data(qrng_data, prng_data)

    # Define the output filename for the results
    output_name = f'{total_qubits}qb_qrng_prng_output.txt'

    # Write the combined data to a text file
    write_data_to_file(combined_data, output_name)

Running circuit with 127 qubits.
[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>]


/tmp/ipykernel_7899/2694583104.py:45: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = device.run(transpiled_circuit, shots=shot_count, memory=True)


Job ID: cw55cnt6f0t000872bw0
Running circuit with 127 qubits.
[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>]


/tmp/ipykernel_7899/2694583104.py:45: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = device.run(transpiled_circuit, shots=shot_count, memory=True)


Job ID: cw55crkqg250008cw08g
Running circuit with 127 qubits.
[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>]


/tmp/ipykernel_7899/2694583104.py:45: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = device.run(transpiled_circuit, shots=shot_count, memory=True)


Job ID: cw55cv3xa9wg0087vn10
Running circuit with 119 qubits.
[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>]


/tmp/ipykernel_7899/2694583104.py:45: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = device.run(transpiled_circuit, shots=shot_count, memory=True)


Job ID: cw55cxvqg250008cw090


KeyboardInterrupt: 